# 'monGPT

This program builds plausible gen 8 ou teams using minGPT

In [1]:
import os

import torch
from mongpt_dataset import PokemonDataset, RandomizeTeamWrapper, InputTeamWrapper, GPTTeamWrapper
from team_generator import fill_pokemon_team

from mongpt.model import GPT
from mongpt.trainer import Trainer

In [2]:
save_dir = 'models'
os.makedirs(save_dir, exist_ok=True)

# load the dataset
dataset = PokemonDataset('dataset/replays')
dataset = RandomizeTeamWrapper(dataset)
dataset = InputTeamWrapper(dataset, num_input_pokemon=[1,2,3])
dataset = GPTTeamWrapper(dataset)

Loading replays: 932it [00:00, 7234.61it/s]            


In [3]:
# create the minGPT model

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = dataset.get_vocab_size()
model_config.block_size = dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.11M


In [4]:
# create the trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 2000
trainer = Trainer(train_config, model, dataset)

running on device cuda


In [5]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

iter_dt 0.00ms; iter 0: train loss 6.02693
iter_dt 7.00ms; iter 100: train loss 4.98562
iter_dt 7.00ms; iter 200: train loss 4.79757
iter_dt 7.00ms; iter 300: train loss 4.74014
iter_dt 8.00ms; iter 400: train loss 4.57710
iter_dt 7.00ms; iter 500: train loss 4.49076
iter_dt 7.00ms; iter 600: train loss 4.29716
iter_dt 7.00ms; iter 700: train loss 4.37394
iter_dt 7.00ms; iter 800: train loss 4.36536
iter_dt 7.00ms; iter 900: train loss 4.34426
iter_dt 7.00ms; iter 1000: train loss 4.30566
iter_dt 1408.09ms; iter 1100: train loss 4.20168
iter_dt 7.00ms; iter 1200: train loss 3.90369
iter_dt 8.00ms; iter 1300: train loss 4.18152
iter_dt 6.00ms; iter 1400: train loss 4.12491
iter_dt 7.00ms; iter 1500: train loss 4.14398
iter_dt 10.00ms; iter 1600: train loss 4.19459
iter_dt 7.00ms; iter 1700: train loss 4.02980
iter_dt 8.00ms; iter 1800: train loss 4.07113
iter_dt 6.00ms; iter 1900: train loss 4.01500


In [6]:
# save the model
torch.save(model.state_dict(), os.path.join(save_dir, 'monGPT.pt'))

# Evaluation

Now, lets see how well the model can generate teams

In [7]:
# generate a few teams
team = ['Blissey', 'Garchomp']
print(fill_pokemon_team(team, model))

['Blissey', 'Garchomp', 'Rillaboom', 'Heatran', 'Magnezone', 'Corviknight']


In [8]:
team = ['Magnezone', 'Tapu Lele', 'Ferrothorn']
print(fill_pokemon_team(team, model))

['Magnezone', 'Tapu Lele', 'Ferrothorn', 'Ditto', 'Blissey', 'Mew']


In [9]:
team = ['Venusaur']
print(fill_pokemon_team(team, model))

['Venusaur', 'Slowking-Galar', 'Corviknight', 'Urshifu-Rapid-Strike', 'Blissey', 'Landorus-Therian']


In [10]:
team = []
print(fill_pokemon_team(team, model))

['Ferrothorn', 'Zapdos', 'Pelipper', 'Landorus-Therian', 'Urshifu-Rapid-Strike', 'Rotom-Wash']
